In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 15 08:37:23 2023

@author: Meva
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import random
import pickle

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/ArchivosCSV/ArchivosAuxiliares/')
import market_data
import portfolio
import assert_frame

"""
Explicación de la Tarea-Examen 04.

Dada una lista de activos o rics, tu gerente de portafolios quiere que calcules todas
las métricas estadísticas posibles de todos los portafolios que hemos visto en clase.

En esta ocasión los inputs son 3 listas constantes:
1. La lista de rics, es decir los activos que constituirán a los portafolios
2. La lista de portafolios, que contiene a todos los portafolios especiales que hemos
calculado, incluyendo "volatility-weighted"
3. La lista de métricas de portafolios, que contiene casi todas las métricas disponibles
en la clase portfolio.output

Lo que ella espera que le regrese tu código es:
1. Un dataframe, es decir una tabla
2. La primera columna se llama 'portfolio' y contiene la lista de portafolios.
3. Las siguientes columnas son las métricas de portafolios.
4. El redondeo es a 6 decimales.

Observa en la lista portfolios los últimos dos portafolios especiales:

El portafolios "markowitz-none" es el portafolios de Markowitz con target_return no
especificado , es decir target_return = None, lo cual por default se traduce en
el mismo rendimiento del portafolios equi-ponderado.

El portafolios "markowitz-target" va a tener un target_return específico.
Si r el rendimiento anualizado del portafolios equiponderado entonces target_return
será r redondeado a 3 decimales más 0.1% (o 10 bps):
target_return = np.round(r,3) + 0.001

"""

portfolios = ['min-variance-l1',\
            'min-variance-l2',\
            'equi-weight',\
            'long-only',\
            'volatility-weighted',\
            'markowitz-none',\
            'markowitz-target']

metrics = ['target_return',\
           'mean_annual',\
           'volatility_annual',\
           'sharpe_ratio',\
           'var_95',\
           'skewness',
           'kurtosis']


# función examen para los alumnos
def generate_dataframe_distributions(rics, portfolios, metrics):
    df = pd.DataFrame()
    decimals=6
    notional = 15
    port_mgr = portfolio.manager(rics, notional)
    port_mgr.compute_covariance()
    ports= []

    for p in portfolios:
        port= None
        if p == "markowitz-none":
          port = port_mgr.compute_portfolio("markowitz")

        elif p == "markowitz-target":
          aux = port_mgr.compute_portfolio("equi-weight")
          aux.compute_stats
          r = aux.return_annual
          target_return = np.round(r,3) + 0.001
          port = port_mgr.compute_portfolio("markowitz",target_return)

        else: port = port_mgr.compute_portfolio(p)

        port.compute_stats
        stat = [p,port.target_return,\
                np.round(port.return_annual,decimals),\
                np.round(port.volatility_annual,decimals),\
                np.round(port.sharpe_ratio,decimals),\
                np.round(port.var_95,decimals),\
                np.round(port.skewness,decimals),\
                np.round(port.kurtosis,decimals)]

        ports.append(stat)
    df = pd.DataFrame(ports)
    metrics= ["portfolio"]+metrics
    df.columns=metrics

    return df


# evaluation loop
universe = ['^SPX','^IXIC','^MXX','^STOXX','^GDAXI','^FCHI','^VIX',\
            'XLK','XLF','XLV','XLE','XLC','XLY','XLP','XLI','XLB','XLRE','XLU',\
            'SPY','EWW',\
            'IVW','IVE','QUAL','MTUM','SIZE','USMV',\
            'AAPL','MSFT','NVDA','AMZN','GOOG','META','NFLX',\
            'BRK-B','JPM','V','MA','BAC','MS','GS','BLK',\
            'LLY','JNJ','PG','MRK','ABBV','PFE',\
            'BTC-USD','ETH-USD','SOL-USD','USDC-USD','USDT-USD','DAI-USD',\
            'EURUSD=X','GBPUSD=X','CHFUSD=X','SEKUSD=X','NOKUSD=X','JPYUSD=X','MXNUSD=X']
nb_tests = 50
instances = []
with open('/content/drive/MyDrive/ArchivosCSV/ArchivosAuxiliares/examen_04_instances.pickle','rb') as f:
    instances = pickle.load(f)
instances = random.sample(instances, nb_tests)
n = 0
final_note = 0.0
for instance in instances:
    n += 1
    print('#######')
    print('Test ' + str(n))
    print('#######')
    rics = instance[0]
    df_teacher = instance[1]
    print('rics = ' + str(rics))
    df_student = generate_dataframe_distributions(rics, portfolios, metrics)
    note = assert_frame.compare_dataframes(df_teacher, df_student)
    final_note += note
    print('---')
final_note /= nb_tests
print('#######')
print('Your final note is ' + str(final_note))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
#######
Test 1
#######
rics = ['GOOG', 'XLE', 'AMZN', 'EWW', 'V', 'MRK', 'AAPL', 'MTUM', 'SOL-USD', 'QUAL', '^MXX']
---
 Comparing dataframes df_teacher vs df_student
---
Note = 10.0
---
#######
Test 2
#######
rics = ['XLC', '^IXIC', 'NVDA', 'EURUSD=X', 'XLI', 'DAI-USD', '^MXX', 'IVE', 'META', 'ABBV', 'GOOG', 'EWW']
---
 Comparing dataframes df_teacher vs df_student
---
Note = 10.0
---
#######
Test 3
#######
rics = ['^SPX', 'SIZE', 'MXNUSD=X', 'XLF', 'BRK-B', 'EURUSD=X', 'JNJ']
---
 Comparing dataframes df_teacher vs df_student
---
Note = 10.0
---
#######
Test 4
#######
rics = ['SIZE', 'IVW', 'DAI-USD', 'USDC-USD', 'SEKUSD=X', 'XLY', 'XLC', 'EWW', 'JNJ']
---
 Comparing dataframes df_teacher vs df_student
---
Note = 10.0
---
#######
Test 5
#######
rics = ['NVDA', '^VIX', '^GDAXI', 'IVE', 'XLF', 'JNJ', 'ABBV', 'AMZN', 'SPY', '^SPX', 'EURUSD=X', 'MS', 'XLB', 'BL